# End To End

This notebook features some examples on some advanced end to end pipeline that really leverage NumerBlox's power. Consider this a testing ground on how well it integrates with sklearn and associated libraries.

In [1]:
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeClassifier
from sklego.preprocessing import ColumnSelector
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import make_pipeline, make_union
from sklearn.compose import TransformedTargetRegressor

from numerblox.preprocessing import GroupStatsPreProcessor
from numerblox.meta import CrossValEstimator, make_meta_pipeline
from numerblox.ensemble import NumeraiEnsemble, PredictionReducer
from numerblox.neutralizers import FeatureNeutralizer

## 0. Get data

In [2]:
from numerblox.numerframe import create_numerframe

df = create_numerframe("../tests/test_assets/train_int8_5_eras.parquet")

In [3]:
X, y = df.get_feature_target_pair(multi_target=False)
fncv3_cols = df.get_fncv3_features.columns.tolist()
eras = df.get_era_data
features = df.get_feature_data

## 1. Weighted XGBoost ensemble pipeline with feature neutralization.

In [4]:
# !pip install xgboost sklego

In [5]:
# Preprocessing
gpp = GroupStatsPreProcessor(groups=['sunshine', 'rain'])
fncv3_selector = ColumnSelector(fncv3_cols)

preproc_pipe = make_union(gpp, fncv3_selector)

# Model
xgb = XGBRegressor()
cve = CrossValEstimator(estimator=xgb, cv=TimeSeriesSplit(n_splits=5))
fn = FeatureNeutralizer(proportion=0.5)
ens = NumeraiEnsemble()
full_pipe = make_meta_pipeline(preproc_pipe, cve, ens, fn)
full_pipe

MetaPipeline(steps=[('featureunion',
                     FeatureUnion(transformer_list=[('groupstatspreprocessor',
                                                     GroupStatsPreProcessor(groups=['sunshine',
                                                                                    'rain'])),
                                                    ('columnselector',
                                                     ColumnSelector(columns=['feature_honoured_observational_balaamite',
                                                                             'feature_polaroid_vadose_quinze',
                                                                             'feature_untidy_withdrawn_bargeman',
                                                                             'feature_genuine_kyphotic_trehala',
                                                                             'feature_unenthr...
                                                              max_depth=None,
                                                              min_child_weight=None,
                                                              missing=nan,
                                                              monotone_constraints=None,
                                                              n_estimators=100,
                                                              n_jobs=None,
                                                              num_parallel_tree=None,
                                                              random_state=None,
                                                              reg_alpha=None,
                                                              reg_lambda=None,
                                                              scale_pos_weight=None,
                                                              subsample=None,
                                                              tree_method=None,
                                                              validate_parameters=None,
                                                              verbosity=None))),
                    ('numeraiensemble', NumeraiEnsemble()),
                    ('featureneutralizer', FeatureNeutralizer())])

In [6]:
# Train full model
full_pipe.fit(X, y, numeraiensemble__eras=eras, featureneutralizer__eras=eras, featureneutralizer__features=features);

In [7]:
# Unneutralized predictions
full_pipe[:-1].predict(X, eras=eras)[:5]

array([[0.15709459],
       [0.84744208],
       [0.19782819],
       [0.82215251],
       [0.63055019]])

In [8]:
# End to end predictions
preds = full_pipe.predict(X=X, features=features, eras=eras)
preds[:5]

array([[0.36577675],
       [0.64867036],
       [0.38364433],
       [0.5980937 ],
       [0.62150554]])

## 2. Multiclass Classification Ensemble

In [9]:
model = DecisionTreeClassifier()
crossval1 = CrossValEstimator(estimator=model, cv=TimeSeriesSplit(n_splits=3), predict_func='predict_proba')
pred_rud = PredictionReducer(n_models=3, n_classes=5)
ens2 = NumeraiEnsemble(donate_weighted=True)
neut2 = FeatureNeutralizer(proportion=0.5)
pipe2 = make_pipeline(preproc_pipe, crossval1, pred_rud, ens2, neut2)
full_pipe = TransformedTargetRegressor(pipe2, func=lambda x: (x * 4).astype(int), inverse_func=lambda x: x)

In [10]:
full_pipe

TransformedTargetRegressor(func=<function <lambda> at 0x7efbc6f2be50>,
                           inverse_func=<function <lambda> at 0x7efbc6f2b790>,
                           regressor=Pipeline(steps=[('featureunion',
                                                      FeatureUnion(transformer_list=[('groupstatspreprocessor',
                                                                                      GroupStatsPreProcessor(groups=['sunshine',
                                                                                                                     'rain'])),
                                                                                     ('columnselector',
                                                                                      ColumnSelector(columns=['feature_honoured_observational_balaamite'...
                                                                                                              'feature_inconsiderate_unbooted_ricer', ...]))])),
                                                     ('crossvalestimator',
                                                      CrossValEstimator(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                                                                        estimator=DecisionTreeClassifier(),
                                                                        predict_func='predict_proba')),
                                                     ('predictionreducer',
                                                      PredictionReducer(n_classes=5,
                                                                        n_models=3)),
                                                     ('numeraiensemble',
                                                      NumeraiEnsemble(donate_weighted=True)),
                                                     ('featureneutralizer',
                                                      FeatureNeutralizer())]))

In [11]:
preds = full_pipe.fit(X, y, numeraiensemble__eras=eras)
preds = full_pipe.predict(X, eras=eras)
preds[:5]

TypeError: transform() missing 1 required positional argument: 'eras'